In [1]:
__file__=''
import os,sys
import pandas as pd
sys.path.append(os.path.join(os.path.dirname(__file__),'../LIB/'))
sys.path.append(os.path.join(os.path.dirname(__file__),'../../../../automl/automl_libs/'))
from env import FILE
from sklearn.feature_extraction import FeatureHasher
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
from tqdm import tqdm
from scipy.sparse import csc_matrix, csr_matrix, hstack
import time
import numpy as np
import pickle

from keras.layers import Dense,Input,LSTM,Bidirectional,Activation,Conv1D,GRU,CuDNNGRU,Flatten,BatchNormalization,CuDNNLSTM,Activation,BatchNormalization
from keras.callbacks import Callback
from keras.layers import Dropout,Embedding,GlobalMaxPooling1D, MaxPooling1D, Add, Flatten
from keras.preprocessing import text, sequence
from keras.layers import GlobalAveragePooling1D, GlobalMaxPooling1D, concatenate, SpatialDropout1D
from keras import initializers, regularizers, constraints, optimizers, layers, callbacks
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras.models import Model
from keras.optimizers import Adam
from sklearn.utils import shuffle
from keras.preprocessing import text, sequence
from sklearn.metrics import log_loss,roc_auc_score
from keras.regularizers import l1
from keras.regularizers import l2
from keras.regularizers import l1_l2
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, StratifiedKFold
import re
from keras import backend as K
import time
import gc
# To get learning rate


/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
train = pd.read_pickle(FILE.train_ori.value)
print('train shape is: {}'.format(train.shape))
test = pd.read_pickle(FILE.test_ori.value)
print('test shape is: {}'.format(test.shape))

# X = pd.concat([train.drop(['click'],axis=1),test])
X = pd.concat([train,test],sort=False)
print(X.shape)

X_clean = pd.read_csv('../../data/original/cleaned_data_price_final.csv')

X_shiyi = pd.read_pickle(FILE.shiyi_fillna_ori.value)
print(X_shiyi.shape)

X = X.merge(X_shiyi[['time_hour','instance_id']],how='inner',on='instance_id')
print(X.shape)
assert np.sum(X_clean['instance_id'].values != X['instance_id'].values) == 0
X['model'] = X_clean['model_new'].values
X['price'] = X_clean['price'].values
# X['make'] = X_clean['make_new'].values
# X.drop('carrier',axis=1,inplace=True)

ignore_columns = ['instance_id','time','click'] + ['creative_is_js', 'creative_is_voicead', 'app_paid']

train shape is: (1001650, 35)
test shape is: (40024, 34)
(1041674, 35)
(1041674, 45)
(1041674, 36)


# PB 4236

In [3]:
# need_process_col = list(set(X.columns) - set(ignore_columns))
# X_ = X[need_process_col].copy()


# doc_col = ['user_tags','model']
# non_doc_col = [f for f in need_process_col if f not in doc_col]
# # doc_col = doc_col + []
# # non_doc_col = non_doc_col+['user_tags']
# counter = 0

# X_doc = X[doc_col].copy()
# for col in tqdm(non_doc_col):
#     X_[col] = le.fit_transform(X_[col].astype(str))
#     X_[col] = col + '_'+X_[col].astype(str)
    
# for col in tqdm(doc_col):
#     X_doc[col] = X_doc[col].astype(str)
    
    

    

# Combine columns

In [4]:
# def comb_fe(X,cols,sep=' '):
#     ret = X[cols[0]].astype(str).copy()
#     for col in cols[1:]:
#         ret = ret + sep + X[col].astype(str)
#     return ret.values

def process_slot(x):
    x = re.sub(r'[-_]',' ',x)
    x = x.split(' ')
    ret = ['p{}_'.format(c)+x[c] for c in range(len(x))]
    ret = ' '.join(ret)
    return ret

def comb_fe(X,cols,sep=' '):
    def add_col_name(x,colName,splitter=' ' ,sep=' '):
        ret = [colName+'_'+each for each in x.split(splitter)]
        return sep.join(ret)
    ret = pd.DataFrame()
    if cols[0] == 'user_tags':
        ret['comb'] = X['user_tags'].astype(str).apply(add_col_name,colName='userTags',splitter=',',sep=sep)
    else:
        if cols[0] in original_name_col:
            ret['comb'] = X[cols[0]].astype(str).copy()
        else:
            ret['comb'] = X[cols[0]].astype(str).apply(add_col_name,colName=cols[0],splitter=' ',sep=sep)

    for col in tqdm(cols[1:]):
        if col == 'user_tags':
            ret['new_feature'] = X['user_tags'].astype(str).apply(add_col_name,colName='userTags',splitter=',',sep=sep)
        else:
            if col in original_name_col:
                ret['new_feature'] = X[col].astype(str).copy()
            else:
                ret['new_feature'] = X[col].astype(str).apply(add_col_name,colName=col,splitter=' ',sep=sep)
        ret['comb'] =( ret['comb'] + sep + ret['new_feature']).values
    return ret['comb'].values

processed_col = []
original_name_col = ['model','make','os','osv']
# cob_col = [['model','make','os','osv']]
# cob_col = [['model','make','os','osv'],['model','app_id','inner_slot_id','creative_width', 'creative_height']]
# cob_col = [['model', 'osv']]
cob_col=[]
doc_col=['user_tags','model']
# doc_col=['user_tags']
X_doc = X[doc_col].copy()
processed_col.extend(doc_col)
for each in cob_col:
    feature_name = '_'.join(each)
    processed_col.extend(each)
    doc_col.append(feature_name)
    X_doc[feature_name] = comb_fe(X,each)
    
non_doc_col = list(set(X.columns) - set(ignore_columns) - set(processed_col))
X_ = X[non_doc_col].copy()

for col in tqdm(non_doc_col):
    X_[col] = le.fit_transform(X_[col].astype(str))
    X_[col] = col + '_'+X_[col].astype(str)
    
for col in tqdm(doc_col):
    if col=='inner_slot_id':
        X_doc[col] = X_doc[col].astype(str).apply(process_slot)
    else:
        X_doc[col] = X_doc[col].astype(str)

100%|██████████| 2/2 [00:00<00:00,  6.77it/s]


In [5]:
train_index = pickle.load(open(FILE.train_index.value,'rb'))
holdout_index = pickle.load(open(FILE.holdout_index.value,'rb'))

In [6]:
num_folds = 7
seed = 1001
train_index_list = []
val_index_list = []
folds = StratifiedKFold(n_splits= num_folds, shuffle=True, random_state=seed)
for t,v in folds.split(train.loc[train_index],train.loc[train_index,'click']):
    train_index_list.append(train.loc[train_index].iloc[t].index)
    val_index_list.append(train.loc[train_index].iloc[v].index)
    
check = []
for i in val_index_list:
    check.extend(list(i))
assert len(set(check+list(holdout_index))) == len(train)

In [7]:
train_fold_y = {}
val_fold_y = {}
holdout_y = train.loc[holdout_index,'click'].values
for fold in range(num_folds):
    train_fold_y[fold] = train.loc[train_index_list[fold],'click'].values
    val_fold_y[fold] = train.loc[val_index_list[fold],'click'].values

In [8]:
info_dict = {}
train_all_dict = {}
train_fold_dict = {}

val_fold_dict = {}

holdout_input_dict = {}
test_input_dict = {}
maxlen = 1
prefix_input_nonDoc = 'input_'
prefix_input_Doc = 'input_rnn_'
for col in tqdm(non_doc_col):

    maxlen = 1
    tok=text.Tokenizer(num_words=X_[col].nunique(),lower=False,filters='@')
    tok.fit_on_texts(list(X_[col]))
    info_dict.update({prefix_input_nonDoc+col:{'tok':tok}})
    t = tok.texts_to_sequences(list(X_[col].iloc[:1001650].values))
    te = tok.texts_to_sequences(list(X_[col].iloc[1001650:].values))
    train_all_dict[prefix_input_nonDoc+col] = sequence.pad_sequences(t,maxlen=maxlen)
    test_input_dict[prefix_input_nonDoc+col] = sequence.pad_sequences(te,maxlen=maxlen)
    holdout_input_dict[prefix_input_nonDoc+col] = train_all_dict[prefix_input_nonDoc+col][list(holdout_index)]
    
    for fold in range(num_folds):
        if train_fold_dict.get(fold) is None:
            train_fold_dict[fold] = {}
            val_fold_dict[fold] = {}
        train_fold_dict[fold].update({prefix_input_nonDoc+col: train_all_dict[prefix_input_nonDoc+col][list(train_index_list[fold])]})
        val_fold_dict[fold].update({prefix_input_nonDoc+col:train_all_dict[prefix_input_nonDoc+col][list(val_index_list[fold])]})
        
sequence_size_dict = {}
for col in tqdm(doc_col):
    if col == 'user_tags':
        maxlen = 50
        tok=text.Tokenizer(num_words=X_doc[col].nunique(),lower=False,filters=',')
        
    elif col == 'model':
        maxlen = 10
        tok=text.Tokenizer(num_words=X_doc[col].nunique(),lower=False)
    else:
        maxlen = 25
        tok=text.Tokenizer(num_words=X_doc[col].nunique(),lower=False,filters=' ')
    tok.fit_on_texts(list(X_doc[col]))
    info_dict.update({prefix_input_Doc+col:{'tok':tok}})
    sequence_size_dict[col] = maxlen
    t = tok.texts_to_sequences(list(X_doc[col].iloc[:1001650].values))
    te = tok.texts_to_sequences(list(X_doc[col].iloc[1001650:].values))
    train_all_dict[prefix_input_Doc+col] = sequence.pad_sequences(t,maxlen=maxlen)
    test_input_dict[prefix_input_Doc+col] = sequence.pad_sequences(te,maxlen=maxlen)
    holdout_input_dict[prefix_input_Doc+col] = train_all_dict[prefix_input_Doc+col][list(holdout_index)]
    
    for fold in range(num_folds):
        if train_fold_dict.get(fold) is None:
            train_fold_dict[fold] = {}
            val_fold_dict[fold] = {}
        train_fold_dict[fold].update({prefix_input_Doc+col: train_all_dict[prefix_input_Doc+col][list(train_index_list[fold])]})
        val_fold_dict[fold].update({prefix_input_Doc+col:train_all_dict[prefix_input_Doc+col][list(val_index_list[fold])]})

100%|██████████| 2/2 [00:44<00:00, 22.47s/it]


# Build NN model only use model

# 0.41479

In [9]:
Num_filter = 25
def get_nn_model(cols,doc_cols=[],numu_cols=[]):
    """
    cols, used to do ebd and dense layers
    doc_cols: used to do rnn
    there can be overlaps
    """
    input_list = []
    concat_list = []
    numu_list = []
    for col in cols:
        max_feature = len(info_dict[prefix_input_nonDoc+col]['tok'].index_word)
        embed_size = int(np.log2(max_feature)/np.log2(1.5))
        if embed_size< 2:
            embed_size = 2
        cur_input = Input(shape=(1, ),name = prefix_input_nonDoc+col)
        
       
        embed_layer = Embedding(max_feature,
                            embed_size,
                            input_length=1,
                            trainable=True,
                            embeddings_regularizer=l2(0.0005),
                            name='ebd_'+col)(cur_input)
        embed_layer = SpatialDropout1D(0.5)(embed_layer)
        x = Flatten()(embed_layer)
        input_list.append(cur_input)
        concat_list.append(x)
    for col in doc_cols:
        max_feature = len(info_dict[prefix_input_Doc+col]['tok'].index_word)
        embed_size = int(np.log2(max_feature)/np.log2(1.5))
        if embed_size< 2:
            embed_size = 2
        input_shape = sequence_size_dict[col]
        cur_input = Input(shape=(input_shape, ),name = prefix_input_Doc+col)
        embed_layer = Embedding(max_feature,
                            embed_size,
                            input_length=input_shape,
                            trainable=True,
                            embeddings_regularizer=l2(0.0005),
                            name='ebd_rnn_'+col)(cur_input)
        x = SpatialDropout1D(0.5)(embed_layer)
        x = Bidirectional(CuDNNGRU(25, return_sequences=True))(x)
        x = Conv1D(Num_filter, kernel_size = 3, padding = "valid", kernel_initializer = "glorot_uniform")(x)
        x_aveP = GlobalAveragePooling1D()(x)
        x_maxP = GlobalMaxPooling1D()(x)
        x = concatenate([x_aveP,x_maxP],name='pooling_{}'.format(col))
        input_list.append(cur_input)
        concat_list.append(x)

    if len(numu_cols) > 0:
        print('add numu...')
        nu_shape = len(numu_cols)
        cur_input = Input(shape=(nu_shape, ),name = prefix_input_nu)
        x = BatchNormalization()(cur_input)
        x = Dense(128, activation='relu')(x)
        x = Dropout(0.2)(x)
        input_list.append(cur_input)
        numu_list.append(x)
       
    if len(concat_list) > 1:
        x = concatenate(concat_list)
#     x = BatchNormalization()(x)
    

    if len(numu_list)>0:
        x = concatenate([x]+numu_list)
#         x = BatchNormalization()(x)
        
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.2)(x)
    
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.2)(x)

    preds = Dense(1, activation="sigmoid")(x)
    model = Model(input_list, preds)
    model.compile(loss='binary_crossentropy',optimizer=Adam(),metrics=['accuracy'])    
    return model


In [10]:
def train_each_fold(input_train_dict,input_val_dict,y_train,y_val,cols,doc_col=[],tolerance=30,train_batch_size=5000):
    model = get_nn_model(cols,doc_col)
    cur_to = 0
    best_logloss = None
    best_weights = None
    count = 0
    base_lr = 0.001
    while True:
        model.fit(input_train_dict, y_train, 
                  batch_size=train_batch_size, 
                  epochs=1,
                  verbose=1,
                  shuffle=True,
                  )
        preds = model.predict(input_val_dict,5000,verbose=1)
        logloss = log_loss(y_val,preds)
        roc = roc_auc_score(y_val,preds)
        print(logloss)
        print(roc)
        if best_logloss is None:
            best_logloss = logloss
            best_weights = model.get_weights()
        else:
            if best_logloss > logloss:
                best_logloss = logloss
                best_weights = model.get_weights()
                cur_to = 0
            else:
                cur_to +=1
        if cur_to == tolerance:
            break
        print('best logloss is: {}'.format(best_logloss))
        print('remainning trial is: {}/{}'.format(cur_to,tolerance))
        print('total epoch trained: {}'.format(count))
        count+=1
    model.set_weights(best_weights)
    return model
    

In [11]:
def nn_K_fold(train_fold_dict,
              val_fold_dict,
              train_fold_y,
              val_fold_y,
              test_input_dict,
              val_index_list,
              train_df,
              test_df,
              pred_col_name = 'predicted_score',
              holdout_input_dict=None,
              holdout_y=None,
              holdout_index_list=None,
              nondoc_cols=[],
              doc_cols=[],
              tolerance=30,
              train_batch_size=5000,
              preds_batch=5000):
    """
    train_fold_dict: format, key - foldNum, value - nn input 
    train_fold_y: label for train fold, fotmat, key -foldNum, value - label
    val_fold_dict: format, key - foldNum, value - nn input 
    val_fold_y: label for val fold, fotmat, key -foldNum, value - label
    test_input_dict: format, key - foldNum, value - nn input 
    holdout_input_dict: Noneable, if none, not using holdout
    holdout_y: Noneable, label for holdout
    cols: all cols used to do ebd
    doc_cols, cols that are used to do rnn
    val_index_list: cv, going to predict and generate oof
    holdout_index_list: going to predict on each fold
    train_df: 'dataframe which only has id columns, which will be used to store oof prediction'
    test_df: 'dataframe which only has id columns, which will be used to store test prediction'
    """
    train_df = train_df.copy()
    test_df = test_df.copy()
    n_fold = len(train_fold_dict)
    test_preds_list = []
    val_score_list = []
    hold_out_preds_list = []
    holdout_score_list = []
    train_score_list = []
    cv_score_list = []
    train_df[pred_col_name] = np.nan
    test[pred_col_name] = np.nan
    ebd_train = np.zeros((1001650,Num_filter*2))
    holdout_ebd_list = []
    test_ebd_list = []
    
    
    for fold in range(n_fold):
        print('start fold {}...'.format(fold))
        model = train_each_fold(train_fold_dict[fold],
                                val_fold_dict[fold],
                                train_fold_y[fold],
                                val_fold_y[fold],
                                nondoc_cols,
                                doc_col,
                                tolerance=tolerance,
                                train_batch_size=train_batch_size)
        layer_name = 'pooling_user_tags'
        intermediate_layer_model = Model(inputs=model.input,
                                         outputs=model.get_layer(layer_name).output)
        
        train_preds =  model.predict(train_fold_dict[fold],preds_batch,verbose=1)
        val_preds = model.predict(val_fold_dict[fold],preds_batch,verbose=1)
        train_loss = log_loss(train_fold_y[fold],train_preds)
        val_loss = log_loss(val_fold_y[fold],val_preds)
        train_df.loc[val_index_list[fold],pred_col_name] = val_preds
        test_preds = model.predict(test_input_dict,preds_batch,verbose=1)
        test_ebd_list.append(intermediate_layer_model.predict(test_input_dict,preds_batch,verbose=1))
        test_preds_list.append(test_preds)
        val_score_list.append(val_loss)
        train_score_list.append(train_loss)
        print('Fold {} finish! val loss: {}.'.format(fold,val_loss))
        ########## extract ebd
        intermediate_output = intermediate_layer_model.predict(val_fold_dict[fold],preds_batch,verbose=1)
        ebd_train[val_index_list[fold]] = intermediate_output
        ### end
        if holdout_index_list is not None:
            ho_preds = model.predict(holdout_input_dict,preds_batch,verbose=1)
            ho_loss = log_loss(holdout_y,ho_preds)
            ho_roc = roc_auc_score(holdout_y,ho_preds)
            holdout_score_list.append(ho_loss)
            hold_out_preds_list.append(ho_preds)
            print('hold out loss: {}'.format(ho_loss))
            holdout_ebd_list.append(intermediate_layer_model.predict(holdout_input_dict,preds_batch,verbose=1))
        del model
        gc.collect()
        time.sleep(5)
            
    print('finish training... calculating evl matrix')
    test_preds_list = np.array(test_preds_list)
    hold_out_preds_list = np.array(hold_out_preds_list)
    test_preds_final = np.mean(test_preds_list,axis=0)
    cv_score_mean = np.mean(val_score_list)
    train_score_mean = np.mean(train_score_list)
    test_df[pred_col_name] = test_preds_final
    ebd_test = np.mean(test_ebd_list,axis=0)
    print('cv mean is: {}'.format(cv_score_mean))
    if holdout_index_list is not None:
        ho_preds_final = np.mean(hold_out_preds_list,axis=0)
        ho_loss_overall = log_loss(holdout_y,ho_preds_final)
        ho_roc_overall = roc_auc_score(holdout_y,ho_preds_final)
        train_df.loc[holdout_index_list,pred_col_name] = ho_preds_final
        print('holdout loss overall is: {}'.format(ho_loss_overall))
        print('holdout roc overall is: {}'.format(ho_roc_overall))
        ebd_train[holdout_index_list] = np.mean(holdout_ebd_list,axis=0)
        return train_df,test_df,cv_score_mean,ho_loss_overall,train_score_mean,ebd_train,ebd_test
    else:
        return train_df,test_df,cv_score_mean,train_score_mean,ebd_train,ebd_test
    
        

In [12]:
train_df = train[['instance_id']].copy()
test_df = test[['instance_id']].copy()

train_save,test_save,cv_,ho_,ta_ ,ebd_train,ebd_test= nn_K_fold(train_fold_dict,
                                      val_fold_dict,
                                      train_fold_y,
                                      val_fold_y,
                                      test_input_dict,
                                      val_index_list,
                                      train_df,
                                      test_df,
                                      pred_col_name = 'predicted_score',
                                      holdout_input_dict=holdout_input_dict,
                                      holdout_y=holdout_y,
                                      holdout_index_list=holdout_index,
                                      nondoc_cols=non_doc_col,
                                      doc_cols=doc_col,
                                      tolerance=30,
                                      preds_batch=5000)

start fold 0...
Epoch 1/1
137376/137376 [==============================] - 1s 9us/step
0.42030450005487974
0.761284570988974
best logloss is: 0.42030450005487974
remainning trial is: 0/30
total epoch trained: 0
Epoch 1/1
137376/137376 [==============================] - 1s 7us/step
0.41791113470640473
0.7633858967346683
best logloss is: 0.41791113470640473
remainning trial is: 0/30
total epoch trained: 1
Epoch 1/1
137376/137376 [==============================] - 1s 7us/step
0.41955174348956936
0.7636056514790447
best logloss is: 0.41791113470640473
remainning trial is: 1/30
total epoch trained: 2
Epoch 1/1
137376/137376 [==============================] - 1s 7us/step
0.4176311353572292
0.7643998570893603
best logloss is: 0.4176311353572292
remainning trial is: 0/30
total epoch trained: 3
Epoch 1/1
137376/137376 [==============================] - 1s 7us/step
0.41735015250776925
0.7641213477139571
best logloss is: 0.41735015250776925
remainning trial is: 0/30
total epoch trained: 4
Epoch 1

137376/137376 [==============================] - 1s 7us/step
0.4153941580999886
0.7687239487120141
best logloss is: 0.4150905521913728
remainning trial is: 2/30
total epoch trained: 28
Epoch 1/1
137376/137376 [==============================] - 1s 7us/step
0.415419736008314
0.7688251374749757
best logloss is: 0.4150905521913728
remainning trial is: 3/30
total epoch trained: 29
Epoch 1/1
137376/137376 [==============================] - 1s 7us/step
0.41533836276035846
0.7689944346624218
best logloss is: 0.4150905521913728
remainning trial is: 4/30
total epoch trained: 30
Epoch 1/1
137376/137376 [==============================] - 1s 7us/step
0.4155948480751251
0.7690685992418446
best logloss is: 0.4150905521913728
remainning trial is: 5/30
total epoch trained: 31
Epoch 1/1
137376/137376 [==============================] - 1s 7us/step
0.4152139397212538
0.7695700548776874
best logloss is: 0.4150905521913728
remainning trial is: 6/30
total epoch trained: 32
Epoch 1/1
137376/137376 [==========

137376/137376 [==============================] - 1s 7us/step
0.41487127941153357
0.7696181276828234
best logloss is: 0.41475585943830096
remainning trial is: 6/30
total epoch trained: 57
Epoch 1/1
137376/137376 [==============================] - 1s 7us/step
0.41486351744668626
0.7695406272541229
best logloss is: 0.41475585943830096
remainning trial is: 7/30
total epoch trained: 58
Epoch 1/1
137376/137376 [==============================] - 1s 7us/step
0.4146014004294567
0.7704369450262547
best logloss is: 0.4146014004294567
remainning trial is: 0/30
total epoch trained: 59
Epoch 1/1
137376/137376 [==============================] - 1s 7us/step
0.4150583276567927
0.7695311024299776
best logloss is: 0.4146014004294567
remainning trial is: 1/30
total epoch trained: 60
Epoch 1/1
137376/137376 [==============================] - 1s 7us/step
0.41512587791409883
0.769586705591151
best logloss is: 0.4146014004294567
remainning trial is: 2/30
total epoch trained: 61
Epoch 1/1
137376/137376 [======

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



137375/137375 [==============================] - 1s 7us/step
0.41510364305502645
0.7703195476869298
best logloss is: 0.41441473188624717
remainning trial is: 14/30
total epoch trained: 48
Epoch 1/1
137375/137375 [==============================] - 1s 7us/step
0.4153018835778319
0.7697132050237521
best logloss is: 0.41441473188624717
remainning trial is: 15/30
total epoch trained: 49
Epoch 1/1
137375/137375 [==============================] - 1s 7us/step
0.4148856981087432
0.7702629885765955
best logloss is: 0.41441473188624717
remainning trial is: 16/30
total epoch trained: 50
Epoch 1/1
137375/137375 [==============================] - 1s 7us/step
0.41478594493339205
0.7705187471848969
best logloss is: 0.41441473188624717
remainning trial is: 17/30
total epoch trained: 51
Epoch 1/1
137375/137375 [==============================] - 1s 7us/step
0.4149748062675978
0.7705603425594023
best logloss is: 0.41441473188624717
remainning trial is: 18/30
total epoch trained: 52
Epoch 1/1
137375/137375

...

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



137375/137375 [==============================] - 1s 8us/step
0.41710839479869816
0.7652589336374571
best logloss is: 0.41710839479869816
remainning trial is: 0/30
total epoch trained: 5
Epoch 1/1
137375/137375 [==============================] - 1s 7us/step
0.4175480080994331
0.7652553901831645
best logloss is: 0.41710839479869816
remainning trial is: 1/30
total epoch trained: 6
Epoch 1/1
137375/137375 [==============================] - 1s 7us/step
0.41722184380998933
0.765754250680281
best logloss is: 0.41710839479869816
remainning trial is: 2/30
total epoch trained: 7
Epoch 1/1
137375/137375 [==============================] - 1s 8us/step
0.4169515820731476
0.7662091362021354
best logloss is: 0.4169515820731476
remainning trial is: 0/30
total epoch trained: 8
Epoch 1/1
137375/137375 [==============================] - 1s 7us/step
0.41646114455044414
0.7663849419937367
best logloss is: 0.41646114455044414
remainning trial is: 0/30
total epoch trained: 9
Epoch 1/1
137375/137375 [=========

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



0.4154333670540364
0.7681464874656999
best logloss is: 0.4149879638116944
remainning trial is: 20/30
total epoch trained: 60
Epoch 1/1
137375/137375 [==============================] - 1s 7us/step
0.41550826527518103
0.7684769653041555
best logloss is: 0.4149879638116944
remainning trial is: 21/30
total epoch trained: 61
Epoch 1/1
137375/137375 [==============================] - 1s 7us/step
0.4156131946342718
0.7682472381365135
best logloss is: 0.4149879638116944
remainning trial is: 22/30
total epoch trained: 62
Epoch 1/1
137375/137375 [==============================] - 1s 7us/step
0.4151462074499768
0.7684655193260681
best logloss is: 0.4149879638116944
remainning trial is: 23/30
total epoch trained: 63
Epoch 1/1
137375/137375 [==============================] - 1s 7us/step
0.41549738202692055
0.7678233283053726
best logloss is: 0.4149879638116944
remainning trial is: 24/30
total epoch trained: 64
Epoch 1/1
137375/137375 [==============================] - 1s 8us/step
0.4155452448446437

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [13]:
pickle.dump(ebd_train,open('../../data/lgb_feature/train_ebd_50.pkl','wb'))
pickle.dump(ebd_test,open('../../data/lgb_feature/test_ebd_50.pkl','wb'))